In [1]:
from pandas_datareader import data as pdr 
from datetime import date
import yfinance as yf 
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt 
import math
import quandl
import numpy as np

/home/juan/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [47]:
ticker_sp = '^GSPC'
ticker_gold = 'GC=F'
ticker_oil = 'CL=F'
ticker_dax = '^GDAXI'
ticker_nikkei = '^N225'
ticker_ftse = '^FTSE'
ticker_shanghai = '000001.SS'

auth_tok = "Nv1rJgRR7u88iz_dg7Y6"

end_date = "2020-05-1"
start_date = "1950-01-01"

In [48]:
def getGOLDData ():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_GC1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[['Last', 'Volume']]
    data.columns = ["GOLD Adj Close", "GOLD Volume"]
    return data.dropna()

def getSPData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_sp, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["SP500 Adj Close",  "SP500 Volume"]
    return data

def getDAXData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_dax, start=start_date, end=end_date)
    data = data[data.columns[4:6]]
    data.columns = ["DAX Adj Close",  "DAX Volume"]
    return data


def getOILData():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_CL1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[["Last", "Volume"]]
    data.columns=["OIL Adj Close", "OIL Volume"]
    return data.dropna()


def getNIKKEIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_nikkei, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["NIKKEI Adj Close"]
    return data


def getFTSEData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_ftse, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["FTSE Adj Close",  "FTSE Volume"]
    return data

def getSHANGHAIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_shanghai, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SHANGHAI Adj Close"]
    return data

In [49]:
def checkData (data):
    counter = 0
    for index, row in data.iterrows():
        for item in row:
            if (math.isnan(item)):
                counter += 1
                break
    return counter

def normalizeData(data):
    for column in data:
        maxValue = max(data[column])
        data[column] = data[column] / maxValue

In [50]:
def combineData():
    #allData = [getSPData(), getGOLDData(), getDAXData(), getOILData(), getNIKKEIData(), getSHANGHAIData(), getFTSEData()]
    allData = [getSPData()]
    mergedData = pd.concat(allData, axis = 1)
    print("REMOVED: {} DATA POINTS".format(checkData(mergedData)))
    cleanData = mergedData.dropna()
    normalizeData(cleanData)
    return cleanData

In [51]:
data = combineData()

print(data)

day_counter = 0

# Size of prediction
prediction_size = 3

# Number of days in the past
historical_size = 15

# Size of available data.
data_size = len(data) - prediction_size - historical_size

# Size of data alocated to training
training_size = int(0.85*data_size)

# Size of data alocated to testing
testing_size = data_size - training_size

# Number of data elements in a single day
daily_data_size = 2 

x_train = np.zeros((training_size, historical_size, daily_data_size))
y_train = np.zeros((training_size))
x_test  = np.zeros((testing_size, historical_size, daily_data_size))
y_test  = np.zeros((testing_size))

sanity_train = np.zeros((training_size, 3))
sanity_test = np.zeros((testing_size, 3))

[*********************100%***********************]  1 of 1 completed
REMOVED: 0 DATA POINTS
            SP500 Adj Close  SP500 Volume
Date                                     
1950-01-03         0.004920      0.000110
1950-01-04         0.004976      0.000165
1950-01-05         0.005000      0.000223
1950-01-06         0.005015      0.000175
1950-01-09         0.005044      0.000220
...                     ...           ...
2020-04-24         0.837748      0.469132
2020-04-27         0.850075      0.453400
2020-04-28         0.845618      0.495179
2020-04-29         0.868098      0.577864
2020-04-30         0.860101      0.569395

[17696 rows x 2 columns]


In [52]:
day_counter = 0

for index, row in data.iterrows():
    
    if (day_counter == data_size):
        print("100% completed")
        break

    if (day_counter < len(x_train)):
        
        
        for i in range (historical_size):
            for j in range (daily_data_size):
                x_train[day_counter][i][j] = data.iloc[day_counter + i][j]
                
        base_value = data.iloc[day_counter + historical_size - 1]['SP500 Adj Close']
        future_values = []
        for i in range (prediction_size):
            future_values += [data.iloc[day_counter + historical_size + i]['SP500 Adj Close']] 
        average_future = sum(future_values) / len(future_values)
        if ((average_future - base_value) > 0):
            delta = 1
        else:
            delta = 0
            
        y_train[day_counter] = delta
        
        sanity_train[day_counter] = np.array(future_values)
        
        
    else:
        
        for i in range (historical_size):
            for j in range (daily_data_size):
                x_test[day_counter - len(x_train)][i][j] = data.iloc[day_counter + i][j]
                
        base_value = data.iloc[day_counter + historical_size - 1]['SP500 Adj Close']
        future_values = []
        for i in range (prediction_size):
            future_values += [data.iloc[day_counter + historical_size + i]['SP500 Adj Close']] 
        average_future = sum(future_values) / len(future_values)
        if ((average_future - base_value) > 0):
            delta = 1
        else:
            delta = 0
            
        y_test[day_counter - len(y_train)] = delta
        
        sanity_test[day_counter - len(y_train)] = np.array(future_values)
        
    day_counter += 1
    print("{}% completed".format(int(100 * day_counter/data_size)), end = '\r')

100% completed


In [53]:
y_train = y_train.astype('uint8')
y_test  = y_test.astype('uint8') 

In [54]:
# Balance out training data
ones = 0
zeros = 0
for i in y_train:
    if (i == 1):
        ones += 1
    else:
        zeros += 1
balanced_x_train = np.zeros((2*min(ones,zeros), historical_size, daily_data_size))
balanced_y_train = np.zeros((2*min(ones,zeros)))
balanced_sanity_train = np.zeros((2*min(ones,zeros), 3))

# Insert ones
index = 0
for i in range (len(x_train)):
    if (y_train[i] == 1):
        balanced_x_train[index] = x_train[i]
        balanced_y_train[index] = y_train[i]
        balanced_sanity_train[index] = sanity_train[i]
        index += 1
        
    if (index == min(ones, zeros)):
        break
        
# Insert zeros
for i in range (len(x_train)):
    if (y_train[i] == 0):
        balanced_x_train[index] = x_train[i]
        balanced_y_train[index] = y_train[i]
        balanced_sanity_train[index] = sanity_train[i]
        index += 1
        
    if (index == len(balanced_x_train)):
        break

In [55]:
# Balance out testing data
ones = 0
zeros = 0
for i in y_test:
    if (i == 1):
        ones += 1
    else:
        zeros += 1
balanced_x_test = np.zeros((2*min(ones,zeros), historical_size, daily_data_size))
balanced_y_test = np.zeros((2*min(ones,zeros)))
balanced_sanity_test = np.zeros((2*min(ones,zeros), 3))

# Insert ones
index = 0
for i in range (len(x_test)):
    if (y_test[i] == 1):
        balanced_x_test[index] = x_test[i]
        balanced_y_test[index] = y_test[i]
        balanced_sanity_test[index] = sanity_test[i]
        index += 1
        
    if (index == min(ones, zeros)):
        break
        
# Insert zeros
for i in range (len(x_test)):
    if (y_test[i] == 0):
        balanced_x_test[index] = x_test[i]
        balanced_y_test[index] = y_test[i]
        balanced_sanity_test[index] = sanity_test[i]
        index += 1
        
    if (index == len(balanced_x_test)):
        break

In [56]:
ones = 0
zeros = 0
for i in balanced_y_test:
    if (i == 1):
        ones += 1
    else:
        zeros += 1
        
print(ones)
print(zeros)

1114
1114


In [57]:
# Shuffle training data
randomize_training = np.arange(len(balanced_x_train))
np.random.shuffle(randomize_training)
balanced_x_train = balanced_x_train[[randomize_training]]
balanced_y_train = balanced_y_train[[randomize_training]]
balanced_sanity_train = balanced_sanity_train[[randomize_training]]

# Shuffle testing data
randomize_testing = np.arange(len(balanced_x_test))
np.random.shuffle(randomize_testing)
balanced_x_test = balanced_x_test[[randomize_testing]]
balanced_y_test = balanced_y_test[[randomize_testing]]
balanced_sanity_test = balanced_sanity_test[[randomize_testing]]

/home/juan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.
/home/juan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """
/home/juan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np

In [58]:
balanced_y_train = balanced_y_train.astype('uint8')
balanced_y_test  = balanced_y_test.astype('uint8') 